The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

```bash

#first go to home, then make dir
cd $HOME
mkdir students
wget https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv
find ./students -name "LCP_22-23_students.csv"
cp LCP_22-23_students.csv students/LCP_22-23_students.csv

```

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

```bash
touch PoD_students.csv
touch Physics_students.csv
grep "PoD" LCP_22-23_students.csv > PoD_students.csv
grep "Physics" LCP_22-23_students.csv > Physics_students.csv

```

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

```bash
touch initials
for i in {A..Z}
do echo "$i">> initials
grep -c "^$i" LCP_22-23_students.csv >> initials
done

```

1\.d Find out which is the letter with most counts.

```bash

letter="A"
count=0

while read l

do read c

if [ $count -lt $c ]; then letter=$l; count=$c; fi

done <initials

echo "Surname starting with $letter is with most counts ($count)"
```

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

```bash
sed -i '1s/^/#/' LCP_22-23_students.csv 
touch allstud.csv
grep -v "^#" LCP_22-23_students.csv > allstud.csv

sed  '/./=' allstud.csv | sed  '/./N; s/\n/ /' > nallstud.csv

rm allstud.csv

tot=$(grep -c -v "^#" LCP_22-23_students.csv)


touch mod18_stud

for i in $(seq 1 $tot) ;
    do if (( i % 18 == 1)) ; then echo "$i"; grep "^$i " nallstud.csv >> mod18_stud ; fi ;
done

```

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

```bash
cp data.csv ./data.txt
grep -v "^#" data.csv > data.txt
sed -i 's/,//g' data.txt

```

2\.b How many even numbers are there?

```bash
counter=0
while read -r line; do for i in $line; do if [[ $i =~ ^-?[0-9]+$ ]]; then if (( i % 2 == 0 )); then  (( counter+=1 )) ; fi; fi ; done; done < data.txt

echo "$counter"
```


2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

```bash
th=$(echo "100 * sqrt(3) / 2" | bc -l)
count_great=0
count_small=0

while read -r line; do 
    x=$(echo "$line" | cut -d' ' -f1)
    y=$(echo "$line" | cut -d' ' -f2)
    z=$(echo "$line" | cut -d' ' -f3)
    distance=$(echo "sqrt($x*$x + $y*$y + $z*$z)" | bc -l) 
    if [ $(echo "$distance >= $th" | bc -l) -eq 1 ]; then (( count_great++ )); else (( count_small++ )); fi;  
    done < data.txt

echo "Greater are $count_great"
echo "Smaller are $count_small"

```


2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

```bash

n=5

for i in $(seq 1 $n)
    do rm data_$i.txt
    touch data_$i.txt
    while read -r line 
        do for j in $line
            do if [[ $j =~ ^-?[0-9]+$ ]]
                then k=$( echo "scale=3;$j/$i" | bc )
                echo "$k" >> data_$i.txt
                fi
        done
        echo "newline" >> data_$i.txt
    done < data.txt

    sed -i ':a;N;$!ba;s/\n/ /g' data_$i.txt
    sed -i 's/newline/\n/g' data_$i.txt

 done
 
```